In [1]:
import os
import json
import glob
import pandas as pd
import numpy as np
import datetime

In [2]:
# 測站清單，中央氣象局的預報資料是以測站為單位蒐集
stat_list = ['F-D0047-017']

In [3]:
# # 搬移資料夾檔案
# import shutil
# import os
# for stat_id in stat_list:
#     print(stat_id)
#     source = './CWB.3H.Save(2020.05.03~2020.10.06)/{}/'.format(stat_id)
#     destination = './CWB.3H.Save(2020.08.01~2020.10.06)/{}/'.format(stat_id)
    
#     files = os.listdir(source)
#     files = files[len(files)//2:]
#     for file in files:
#         new_path = shutil.move(f"{source}/{file}", destination)

# 打包 3 小時預報資料
### 請先備份並下載原始預報資料

In [4]:
# # 打包預報資料
# def aggregate_data(target_date , stat_list=stat_list, directory='.'):
#     def read_file(stat_id, filename):
#         try:
#             with open(filename) as f:
#                 data = f.read()
#         except UnicodeDecodeError:
#             try:
#                 with open(filename, encoding='utf-8') as f:
#                     data = f.read()
#             except:
#                 print('Unknown error.', stat_id, filename) 
#         except:
#             print('Unknown error.', stat_id, filename)
#         finally:
#             f.close()
#         return data

#     array = []

#     # 依次讀取測站目錄 & 測站目錄裡面的每一個文件
#     for stat_id in stat_list:
#         print('stat:', stat_id)
#         for filename in glob.glob('{}/CWB.3H/{}/*.txt'.format(directory, stat_id)):
# #             
#             file_data = rebuild_crawler_time(filename)
#             start, end = target_date, target_date+datetime.timedelta(days=1)
            
#             if end>file_data>start:

#                 print(file_data)
#                 # 01. 嘗試打開檔案，有可能遇到空白或損毀的檔案，則跳過
#                 file = read_file(stat_id, filename)

#                 # 02. 提取檔案的預報資訊，有可能讀到空白或亂碼檔案，故將該類檔案設為空陣列
#                 # 設定為空陣列的檔案在後續步驟會被跳過
#                 locus = json.loads(file)['records']['locations'][0]

#                 # 03. 檔案內包含多個鄉鎮的預報，逐項讀取
#                 for i in range(len(locus['location'])):
#                     current = locus['location'][i]
#                     # 04. 擷取前 12 小時的預報資料
#                     # 預報內大部分變數包含未來 24 個 3 小時資料(72小時)
#                     # 部分變數包含未來 12 個 6 小時資料(72小時)
#                     for time in range(12):
#                         try:
#                             data = {}

#                             # 05. 檢查當前預報點的屬性是否大於 10 個，否則跳過
#                             forecast = current['weatherElement']
#                             if(len(forecast) < 10):
#                                 continue

#                             # 06. 縣市資訊 & 地區資訊
#                             data['cityName'] = locus['locationsName']
#                             data['locationName'] = current['locationName']
#                             data['geocode'] = current['geocode']
#                             data['lat'] = current['lat']
#                             data['lon'] = current['lon']

#                             # 07. 歷遍預報欄位(11個)
#                             for feature in range(len(forecast)):
#                                 # 賦予屬性名稱，遇到以下欄位做特殊處理
#                                 name = f"{forecast[feature]['elementName']}/{forecast[feature]['description']}"

#                                 if(forecast[feature]['elementName'] == 'PoP6h'):
#                                     try:
#                                         # PoP6h 的預報間隔是 6 小時，故在每 2 筆逐 3 小時資料間插入同樣的值
#                                         values = forecast[feature]['time'][int((time)/2)]['elementValue']
#                                         data[name] = values[0]['value']
#                                     except:
#                                         data[element] = np.nan
#                                 elif(forecast[feature]['elementName'] == 'PoP12h'):
#                                     continue
#                                 elif(forecast[feature]['elementName'] == 'WeatherDescription'):
#                                     continue
#                                 else:
#                                 # 遇到其他(正常欄位)的處理方式
#                                     try:
#                                         # 如果預報欄位有兩個值的場合，需要特別處理
#                                         values = forecast[feature]['time'][time]['elementValue']
#                                         if(len(values) == 2):
#                                             data[name] = values[0]['value']
#                                             data[f'{name}(Unit)'] = values[1]['value']
#                                         elif(len(values)==1):
#                                             data[name] = values[0]['value']
#                                         else:
#                                             print('ERROR.')

#                                     except:
#                                         data[name] = np.nan

#                             # 從欄位「天氣現象」獲取資料的時間戳記
#                             data['datetime'] = forecast[1]['time'][time]['startTime']
#                             data['crawler_time'] = filename
#                             # 將整理完的資料加入陣列
#                             array.append(data)

#                         except:
#                             print(f"Element Error: {filename}, {data['locationName']}, {time+1}, {name}")
    
#     dataframe = pd.DataFrame(array)
#     print('File Length: ', len(dataframe))
    
#     return dataframe

In [5]:
# 打包預報資料
def aggregate_data2(start_date, end_date , stat_list=stat_list, directory='.'):
    def read_file(stat_id, filename):
        try:
            with open(filename) as f:
                data = f.read()
        except UnicodeDecodeError:
            try:
                with open(filename, encoding='utf-8') as f:
                    data = f.read()
            except:
                print('Unknown error.', stat_id, filename) 
        except:
            print('Unknown error.', stat_id, filename)
        finally:
            f.close()
        return data

    array = []

    # 依次讀取測站目錄 & 測站目錄裡面的每一個文件
    for stat_id in stat_list:
        print('stat:', stat_id)
        for filename in glob.glob('{}/CWB.3H/{}/*.txt'.format(directory, stat_id)):
#             
            file_data = rebuild_crawler_time(filename)
#             start, end = target_date, target_date+datetime.timedelta(days=1)
            
            if end_date>file_data>start_date:

                print(file_data)
                # 01. 嘗試打開檔案，有可能遇到空白或損毀的檔案，則跳過
                file = read_file(stat_id, filename)

                # 02. 提取檔案的預報資訊，有可能讀到空白或亂碼檔案，故將該類檔案設為空陣列
                # 設定為空陣列的檔案在後續步驟會被跳過
                if(file[0]=='{'):
                    locus = json.loads(file)['records']['locations'][0]

                # 03. 檔案內包含多個鄉鎮的預報，逐項讀取
                for i in range(len(locus['location'])):
                    current = locus['location'][i]
                    # 04. 擷取前 12 小時的預報資料
                    # 預報內大部分變數包含未來 24 個 3 小時資料(72小時)
                    # 部分變數包含未來 12 個 6 小時資料(72小時)
                    for time in range(12):
                        try:
                            data = {}

                            # 05. 檢查當前預報點的屬性是否大於 10 個，否則跳過
                            forecast = current['weatherElement']
                            if(len(forecast) < 10):
                                continue

                            # 06. 縣市資訊 & 地區資訊
                            data['cityName'] = locus['locationsName']
                            data['locationName'] = current['locationName']
                            data['geocode'] = current['geocode']
                            data['lat'] = current['lat']
                            data['lon'] = current['lon']

                            # 07. 歷遍預報欄位(11個)
                            for feature in range(len(forecast)):
                                # 賦予屬性名稱，遇到以下欄位做特殊處理
                                name = f"{forecast[feature]['elementName']}/{forecast[feature]['description']}"

                                if(forecast[feature]['elementName'] == 'PoP6h'):
                                    try:
                                        # PoP6h 的預報間隔是 6 小時，故在每 2 筆逐 3 小時資料間插入同樣的值
                                        values = forecast[feature]['time'][int((time)/2)]['elementValue']
                                        data[name] = values[0]['value']
                                    except:
                                        data[element] = np.nan
                                elif(forecast[feature]['elementName'] == 'PoP12h'):
                                    continue
                                elif(forecast[feature]['elementName'] == 'WeatherDescription'):
                                    continue
                                else:
                                # 遇到其他(正常欄位)的處理方式
                                    try:
                                        # 如果預報欄位有兩個值的場合，需要特別處理
                                        values = forecast[feature]['time'][time]['elementValue']
                                        if(len(values) == 2):
                                            data[name] = values[0]['value']
                                            data[f'{name}(Unit)'] = values[1]['value']
                                        elif(len(values)==1):
                                            data[name] = values[0]['value']
                                        else:
                                            print('ERROR.')

                                    except:
                                        data[name] = np.nan

                            # 從欄位「天氣現象」獲取資料的時間戳記
                            data['datetime'] = forecast[1]['time'][time]['startTime']
                            data['crawler_time'] = filename
                            # 將整理完的資料加入陣列
                            array.append(data)

                        except:
                            print(f"Element Error: {filename}, {data['locationName']}, {time+1}, {name}")
    
    dataframe = pd.DataFrame(array)
    print('File Length: ', len(dataframe))
    
    return dataframe

In [6]:
# 儲存在 Windows 和 Ubuntu 的檔案名稱不同，在計算下載時間的時候會錯誤
# 透過程式將 2 種命名格式調整至固定格式
def replacer(s, newstring, index, nofail=False):
    # raise an error if index is outside of the string
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string")

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + 1:]
def rebuild_crawler_time(string):
    try:
        new_string = string.replace(".txt", "")
        new_string = new_string.replace('%3A', ':')
        new_string = new_string.replace('_', ':')
        new_string = new_string[len(new_string)-19:len(new_string)-0]
        if(new_string.count(':')>2):
            new_string = replacer(new_string, " ", new_string.find(':'))
        new_string = pd.to_datetime(new_string)
    except:
        new_string = ''
    return new_string

# 整合新舊資料 & 填充缺值
### 包含時間轉換、重新命名、去除重覆資料.....等
### 將整理完成的 3 小時預報和先前的合併
### 另外，根據預測時間，將資料分為 24 小時內和 3 小時內的預報

In [7]:
# 合併新舊預報資料，並儲存至本地資料夾
def merge_with_history(forecast, directory='.'):
    
    forecast_copy = forecast.copy()
    forecast_copy.rename(columns={
        'datetime': 'TIME_TO_INTERVAL', 'crawler_time': 'CrawlerTime',
        'locationName': 'LocationName', 'cityName': 'CityName',
        'lat': 'Latitude', 'lon': 'Longitude',
        'Wx/天氣現象': 'WeatherType', 
        'Wx/天氣現象(Unit)': 'WeatherType(index)',
        'PoP6h/6小時降雨機率': 'PoP6h(pred)',
        'PoP6h': 'PoP6h(pred)',
        'AT/體感溫度': 'ApparentTemperature(pred)',
        'T/溫度': 'Temperature(pred)',
        'CI/舒適度指數': 'ComfortIndex(pred)',
        'CI/舒適度指數(Unit)': 'ComfortIndex(index)',
        'RH/相對濕度': 'RelativeHumidity(pred)',
        'WS/風速': 'WindSpeed(pred)',
        'WS/風速(Unit)': 'WindSpeed(index)',
        'WD/風向': 'WindDirection(pred)',
        'Td/露點溫度': 'DewpointTemperature(pred)'}, inplace=True) 
    forecast_copy['CrawlerTime'] = forecast_copy['CrawlerTime'].apply(lambda x: rebuild_crawler_time(x))
    forecast_copy = build_multiple_lead_time_data(forecast_copy)
#     forecast_copy.to_csv(f'{directory}/CWB.3H/Save/CWB.3H.Merge.Multiple.csv', index=False)
    
    # read history data
    history = pd.read_csv(f'{directory}/CWB.3H/Save/CWB.3H.Merge.Multiple.csv')
    history.rename(columns={
        'datetime': 'TIME_TO_INTERVAL', 'crawler_time': 'CrawlerTime',
        'locationName': 'LocationName', 'cityName': 'CityName',
        'lat': 'Latitude', 'lon': 'Longitude',
        'Wx/天氣現象': 'WeatherType', 
        'Wx/天氣現象(Unit)': 'WeatherType(index)',
        'PoP6h/6小時降雨機率': 'PoP6h(pred)',
        'PoP6h': 'PoP6h(pred)',
        'AT/體感溫度': 'ApparentTemperature(pred)',
        'T/溫度': 'Temperature(pred)',
        'CI/舒適度指數': 'ComfortIndex(pred)',
        'CI/舒適度指數(Unit)': 'ComfortIndex(index)',
        'RH/相對濕度': 'RelativeHumidity(pred)',
        'WS/風速': 'WindSpeed(pred)',
        'WS/風速(Unit)': 'WindSpeed(index)',
        'WD/風向': 'WindDirection(pred)',
        'Td/露點溫度': 'DewpointTemperature(pred)'}, inplace=True)
    
    history['TIME_TO_INTERVAL'] = pd.to_datetime(history['TIME_TO_INTERVAL'])
    rebuild_cwb = pd.concat([forecast_copy, history], axis=0, ignore_index=True)
    rebuild_cwb['TIME_TO_INTERVAL'] = pd.to_datetime(rebuild_cwb['TIME_TO_INTERVAL'])
    rebuild_cwb['CrawlerTime'] = pd.to_datetime(rebuild_cwb['CrawlerTime'])
    rebuild_cwb = rebuild_cwb.sort_values(by=['CityName', 'LocationName', 'TIME_TO_INTERVAL', 'CrawlerTime'], inplace=False)
    
    # 移除重複的預測資料
    rebuild_cwb = rebuild_cwb.drop_duplicates(['TIME_TO_INTERVAL', 'CityName', 'LocationName', 'TimeAhead'], keep="last")
    rebuild_cwb = fill_multiple_lead_time_data(rebuild_cwb)
    # 匯出整理後的資料，檔案類型：CSV
    print(f'history: {len(history)}, merge: {len(rebuild_cwb)}')
    get_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S').replace(':', '%3A')
    
    # 儲存 2 個檔案，分別表示當前最新的整合檔案(merge)，以及紀錄歷史整合結果(file name: get-time)
    rebuild_cwb.to_csv(f'{directory}/CWB.3H/Save/CWB.3H.Merge.Multiple.csv', index=False)
#     rebuild_cwb.to_csv(f'{directory}/CWB.3H/Save/{get_time}.csv', index=False)
    forecast_copy.to_csv(f'{directory}/CWB.3H/Save/CWB.3H.Merge.Multiple(new).csv', index=False)

    return True

In [8]:
# 根據提前預報的時間，將資料分成:「 24小時前 」,「 3小時前」、「 3小時內」
def build_multiple_lead_time_data(data):
    forecast = data.copy()
    forecast['TIME_TO_INTERVAL'] = pd.to_datetime(forecast['TIME_TO_INTERVAL'])
    forecast['CrawlerTime'] = pd.to_datetime(forecast['CrawlerTime'])
    forecast['TimeAhead'] = forecast['TIME_TO_INTERVAL'] - forecast['CrawlerTime']
    forecast['DayAhead'] = forecast['TIME_TO_INTERVAL'].dt.date - forecast['CrawlerTime'].dt.date
    forecast.sort_values(by=['CityName', 'LocationName', 'TIME_TO_INTERVAL', 'TimeAhead'], inplace=True)
    forecast = forecast.reset_index(inplace=False, drop=True)
    
    # 01.
    merge0 = forecast.copy()
    merge0 = merge0.drop_duplicates(['CityName', 'LocationName', 'TIME_TO_INTERVAL'], keep="first")  
    merge0['TimeAhead'] = 0
    print('merge0' ,len(merge0))
    
    # 02.
    merge3 = forecast.copy()
    merge3 = merge3[merge3['TimeAhead']>=datetime.timedelta(hours=3)]
    merge3 = merge3.drop_duplicates(['CityName', 'LocationName', 'TIME_TO_INTERVAL'], keep="first")  
    merge3['TimeAhead'] = 3
    print('merge3', len(merge3))
    
    # 03.
    merge24 = forecast.copy()
    merge24 = merge24[merge24['DayAhead']>=datetime.timedelta(days=1)]
    merge24 = merge24.drop_duplicates(['CityName', 'LocationName', 'TIME_TO_INTERVAL'], keep="first")  
    merge24['TimeAhead'] = 24
    print('merge24', len(merge24))
    
    build = pd.concat([merge24, merge3, merge0], axis=0, ignore_index=True)
    build = build.drop(['DayAhead'], axis=1)
    return build

In [9]:
# 填補不同時間間隔的資料
def fill_multiple_lead_time_data(data):
    forecast = data.copy()
    forecast['TIME_TO_INTERVAL'] = pd.to_datetime(forecast['TIME_TO_INTERVAL'])
    forecast['CrawlerTime'] = pd.to_datetime(forecast['CrawlerTime'])
    forecast = forecast.sort_values(by=['CityName', 'LocationName', 'TIME_TO_INTERVAL', 'CrawlerTime'], inplace=False)
    forecast = forecast.reset_index(inplace=False, drop=True)
    
    # 01.最近抓取的資料
    merge0 = forecast.copy()
    merge0 = merge0.drop_duplicates(['CityName', 'LocationName', 'TIME_TO_INTERVAL'], keep="last")  
    merge0['TimeAhead'] = 0
    print('merge0' ,len(merge0))
    
    # 02.三小時前抓取的
    merge3 = forecast.copy()
    merge3 = merge3[merge3['TimeAhead'].eq(3)]
    filling = merge0[~merge0['TIME_TO_INTERVAL'].isin(merge3['TIME_TO_INTERVAL'].tolist())]
    merge3 = pd.concat([merge3, filling]).reset_index(inplace=False, drop=True)
    merge3['TimeAhead'] = 3
    print('merge3', len(merge3))
    
    # 03.一天以前抓取的
    merge24 = forecast.copy()
    merge24 = merge24[merge24['TimeAhead'].eq(24)]
    filling = merge3[~merge3['TIME_TO_INTERVAL'].isin(merge24['TIME_TO_INTERVAL'].tolist())]
    merge24 = pd.concat([merge24, filling]).reset_index(inplace=False, drop=True)
    merge24['TimeAhead'] = 24
    print('merge24', len(merge24))
    
    build = pd.concat([merge24, merge3, merge0], axis=0, ignore_index=True)
    return build

In [10]:
# 資料整理完成後，清除本地資料夾的原始預報資料
def remove_processed_data(directory='.'):

    for folder in os.listdir(directory + '/CWB.3H/'):
        
        # 依次讀取根目錄下的每一個資料夾下的每一個文件
        if(folder=='Save'):
            print('found save file.')
            continue
            
        for filename in glob.glob('{}/CWB.3H/{}/*.txt'.format(directory, folder)):
            if os.path.isfile(filename):
                os.remove(filename)
            elif os.path.isdir(filename):
                shutil.rmtree(filename,True)
    
    return True

# 主程式

In [11]:
# tdate = pd.to_datetime('2022-05-10')
# # 打包 1 小時預報資料
# forecast = aggregate_data(tdate)
# # 合併新舊預報資料，並儲存至本地資料夾
# merge_with_history(forecast)
# # 資料整理完成後，清除本地資料夾的原始預報資料
# # remove_processed_data()

In [12]:
# start,end = pd.to_datetime('2022-08-13'),pd.to_datetime('2023-01-19')
# forecast = aggregate_data2(start,end)
# merge_with_history(forecast)

stat: F-D0047-017
2022-08-13 15:46:15
2022-08-13 16:46:20
2022-08-13 17:46:24
2022-08-13 18:46:27
2022-08-13 19:46:30
2022-08-13 20:46:33
2022-08-13 21:46:36
2022-08-13 22:46:40
2022-08-13 23:46:43
2022-08-14 00:46:46
2022-08-14 01:46:49
2022-08-14 02:46:53
2022-08-14 03:46:57
2022-08-14 14:15:13
2022-08-14 15:15:17
2022-08-14 16:15:21
2022-08-14 17:15:23
2022-08-14 18:15:27
2022-08-14 19:15:31
2022-08-14 20:15:35
2022-08-14 21:15:38
2022-08-14 22:15:44
2022-08-14 23:15:48
2022-08-15 00:15:51
2022-08-15 23:45:04
2022-08-16 00:45:08
2022-08-16 01:45:11
2022-08-16 02:45:14
2022-08-16 03:45:19
2022-08-16 04:45:23
2022-08-16 05:45:27
2022-08-16 06:45:30
2022-08-16 07:45:33
2022-08-16 08:45:37
2022-08-16 09:45:41
2022-08-16 10:45:45
2022-08-16 11:45:48
2022-08-16 12:45:52
2022-08-16 13:45:56
2022-08-16 14:45:59
2022-08-16 15:46:03
2022-08-16 16:46:07
2022-08-16 17:46:08
2022-08-16 18:46:12
2022-08-16 19:46:15
2022-08-16 20:46:18
2022-08-16 21:46:25
2022-08-16 22:46:28
2022-08-16 23:46:31
20

2022-09-02 01:25:23
2022-09-02 02:25:23
2022-09-02 03:25:23
2022-09-02 04:25:25
2022-09-02 05:25:26
2022-09-02 06:25:26
2022-09-02 07:25:26
2022-09-02 08:25:27
2022-09-02 09:25:27
2022-09-02 10:25:27
2022-09-02 11:25:27
2022-09-02 12:25:28
2022-09-02 13:25:28
2022-09-02 14:25:28
2022-09-02 15:25:28
2022-09-02 16:25:25
2022-09-02 17:25:27
2022-09-02 18:25:27
2022-09-02 19:25:27
2022-09-02 20:25:27
2022-09-02 21:25:28
2022-09-02 22:25:28
2022-09-02 23:25:28
2022-09-03 00:25:28
2022-09-03 01:25:29
2022-09-03 02:25:29
2022-09-03 03:25:29
2022-09-03 04:25:29
2022-09-03 05:25:30
2022-09-03 06:25:30
2022-09-03 07:25:30
2022-09-03 08:25:30
2022-09-03 09:25:32
2022-09-03 10:25:33
2022-09-03 11:25:34
2022-09-03 12:25:34
2022-09-03 13:25:34
2022-09-03 14:25:35
2022-09-03 15:25:35
2022-09-03 16:25:32
2022-09-03 17:25:32
2022-09-03 18:25:34
2022-09-03 19:25:34
2022-09-03 20:25:34
2022-09-03 21:25:34
2022-09-03 22:25:35
2022-09-03 23:25:35
2022-09-04 00:25:35
2022-09-04 01:25:35
2022-09-04 02:25:35


2022-09-17 12:00:17
2022-09-17 13:00:17
2022-09-17 14:00:17
2022-09-17 15:00:17
2022-09-17 16:00:13
2022-09-17 17:00:14
2022-09-17 18:00:14
2022-09-17 19:00:14
2022-09-17 20:00:14
2022-09-17 21:00:14
2022-09-17 22:00:15
2022-09-17 23:00:18
2022-09-18 00:00:18
2022-09-18 01:00:18
2022-09-18 02:00:19
2022-09-18 03:00:19
2022-09-18 04:00:19
2022-09-18 05:00:19
2022-09-18 06:00:19
2022-09-18 07:00:20
2022-09-18 08:00:20
2022-09-18 09:00:20
2022-09-18 10:00:20
2022-09-18 11:00:20
2022-09-18 12:00:20
2022-09-18 13:00:21
2022-09-18 14:00:21
2022-09-18 16:00:41
2022-09-18 17:00:41
2022-09-18 18:00:41
2022-09-18 19:00:42
2022-09-18 20:00:42
2022-09-18 21:00:06
2022-09-18 22:00:06
2022-09-18 23:00:06
2022-09-19 00:00:06
2022-09-19 01:00:06
2022-09-19 02:00:06
2022-09-19 03:00:06
2022-09-19 04:00:06
2022-09-19 05:00:06
2022-09-19 06:00:06
2022-09-19 07:00:06
2022-09-19 08:00:06
2022-09-19 09:00:06
2022-09-19 10:00:06
2022-09-19 11:00:06
2022-09-19 12:00:06
2022-09-19 13:00:06
2022-09-19 14:00:06


2022-10-03 01:59:15
2022-10-03 02:59:15
2022-10-03 03:59:15
2022-10-03 04:59:15
2022-10-03 05:59:15
2022-10-03 06:59:15
2022-10-03 07:59:15
2022-10-03 08:59:15
2022-10-03 09:59:15
2022-10-03 10:59:15
2022-10-03 11:59:15
2022-10-03 12:59:15
2022-10-03 13:59:15
2022-10-03 14:59:15
2022-10-03 15:59:11
2022-10-03 16:59:11
2022-10-03 17:59:11
2022-10-03 18:59:11
2022-10-03 19:59:11
2022-10-03 20:59:11
2022-10-03 21:59:12
2022-10-03 22:59:12
2022-10-03 23:59:12
2022-10-04 00:59:12
2022-10-04 01:59:12
2022-10-04 02:59:12
2022-10-04 03:59:12
2022-10-04 04:59:12
2022-10-04 05:59:12
2022-10-04 06:59:12
2022-10-04 07:59:12
2022-10-04 08:59:12
2022-10-04 09:59:12
2022-10-04 10:59:12
2022-10-04 11:59:12
2022-10-04 12:59:12
2022-10-04 13:59:12
2022-10-04 14:59:12
2022-10-04 15:59:08
2022-10-04 16:59:08
2022-10-04 17:59:08
2022-10-04 18:59:08
2022-10-04 19:59:08
2022-10-04 20:59:08
2022-10-04 21:59:08
2022-10-04 22:59:08
2022-10-04 23:59:08
2022-10-05 00:59:08
2022-10-05 01:59:08
2022-10-05 02:59:08


2022-10-20 06:59:48
2022-10-20 07:59:48
2022-10-20 08:59:48
2022-10-20 09:59:48
2022-10-20 10:59:48
2022-10-20 11:59:48
2022-10-20 12:59:48
2022-10-20 13:59:48
2022-10-20 14:59:48
2022-10-20 15:59:48
2022-10-20 16:59:45
2022-10-20 17:59:45
2022-10-20 18:59:45
2022-10-20 19:59:45
2022-10-20 20:59:45
2022-10-20 21:59:45
2022-10-20 22:59:45
2022-10-20 23:59:45
2022-10-21 00:59:45
2022-10-21 01:59:45
2022-10-21 02:59:45
2022-10-21 03:59:45
2022-10-21 04:59:45
2022-10-21 05:59:45
2022-10-21 06:59:45
2022-10-21 07:59:45
2022-10-21 08:59:45
2022-10-21 09:59:45
2022-10-21 10:59:45
2022-10-21 11:59:45
2022-10-21 12:59:45
2022-10-21 13:59:45
2022-10-21 14:59:45
2022-10-21 15:59:45
2022-10-21 16:59:41
2022-10-21 17:59:41
2022-10-21 18:59:41
2022-10-21 19:59:41
2022-10-21 20:59:41
2022-10-21 21:59:41
2022-10-21 22:59:41
2022-10-21 23:59:41
2022-10-22 00:59:41
2022-10-22 01:59:41
2022-10-22 02:59:41
2022-10-22 03:59:41
2022-10-22 04:59:41
2022-10-22 05:59:41
2022-10-22 06:59:41
2022-10-22 07:59:41


...

2022-12-26 19:00:00
2022-12-26 20:00:00
2022-12-26 21:00:00
2022-12-26 22:00:00
2022-12-26 23:00:00
2022-12-27 00:00:00
2022-12-27 01:00:00
2022-12-27 02:00:00
2022-12-27 03:00:00
2022-12-27 04:00:00
2022-12-27 05:00:00
2022-12-27 06:00:00
2022-12-27 07:00:00
2022-12-27 08:00:00
2022-12-27 09:00:00
2022-12-27 10:00:00
2022-12-27 11:00:00
2022-12-27 12:00:00
2022-12-27 13:00:00
2022-12-27 14:00:00
2022-12-27 15:00:00
2022-12-27 16:00:00
2022-12-27 17:00:00
2022-12-27 18:00:00
2022-12-27 19:00:00
2022-12-27 20:00:00
2022-12-27 21:00:00
2022-12-27 22:00:00
2022-12-27 23:00:00
2022-12-28 00:00:00
2022-12-28 01:00:00
2022-12-28 02:00:00
2022-12-28 03:00:00
2022-12-28 04:00:00
2022-12-28 05:00:00
2022-12-28 06:00:00
2022-12-28 07:00:00
2022-12-28 08:00:00
2022-12-28 09:00:00
2022-12-28 10:00:00
2022-12-28 11:00:00
2022-12-28 12:00:00
2022-12-28 13:00:00
2022-12-28 14:00:00
2022-12-28 15:00:00
2022-12-28 16:00:00
2022-12-28 17:00:00
2022-12-28 18:00:00
2022-12-28 19:00:00
2022-12-28 20:00:00


2023-01-14 21:00:00
2023-01-14 22:00:00
2023-01-14 23:00:00
2023-01-15 00:00:00
2023-01-15 01:00:00
2023-01-15 02:00:00
2023-01-15 03:00:00
2023-01-15 04:00:00
2023-01-15 05:00:00
2023-01-15 06:00:00
2023-01-15 07:00:00
2023-01-15 08:00:00
2023-01-15 09:00:00
2023-01-15 10:00:00
2023-01-15 11:00:00
2023-01-15 12:00:00
2023-01-15 13:00:00
2023-01-15 14:00:00
2023-01-15 15:00:00
2023-01-15 16:00:00
2023-01-15 17:00:00
2023-01-15 18:00:00
2023-01-15 19:00:00
2023-01-15 20:00:00
2023-01-15 21:00:00
2023-01-15 22:00:00
2023-01-15 23:00:00
2023-01-16 00:00:00
2023-01-16 01:00:00
2023-01-16 02:00:00
2023-01-16 03:00:00
2023-01-16 04:00:00
2023-01-16 05:00:01
2023-01-16 06:00:01
2023-01-16 07:00:01
2023-01-16 08:00:01
2023-01-16 09:00:01
2023-01-16 10:00:01
2023-01-16 11:00:01
2023-01-16 12:00:01
2023-01-16 13:00:01
2023-01-16 14:00:01
2023-01-16 15:00:01
2023-01-16 16:00:01
2023-01-16 17:00:00
2023-01-16 18:00:00
2023-01-16 19:00:00
2023-01-16 20:00:00
2023-01-16 21:00:00
2023-01-16 22:00:00


C:\Users\GodZen\Anaconda3\envs\python3.6\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


merge0 36292
merge3 36242
merge24 36242
history: 99884, merge: 108776


True

In [13]:
# def calculate_similar_day(plant_info, latitude, longitude):
#     sdv3 = {}
#     sdv3['CWB'] = int(plant_info['CWB'][0])
#     sdv3['IBM'] = int(plant_info['TWC'][0])
#     sdv3['OWM'] = int(plant_info['OWM'][0])
#     # 氣象局預報
#     keys = ['CWB']
#     for key in keys:
#         print(sdv3['CWB'])
#         if key == 'CWB':
#             weather_data = pd.read_csv('CWB.3H/save/CWB.3H.Merge.Multiple0824.csv')
#         elif key == 'IBM':
#             weather_data = pd.read_csv('WeatherChannel.1H/save/IBM.1H.Merge.Multiple(merge).csv')
#         else:
#             weather_data = pd.read_csv('OpenWeatherMap.3H/save/OWM.3H.Merge.Multiple(merge).csv')
#             weather_data['TIME_TO_INTERVAL'] = pd.to_datetime(weather_data['TIME_TO_INTERVAL'])+datetime.timedelta(hours=1)
#             weather_data['TIME_TO_INTERVAL'] = pd.to_datetime(weather_data['TIME_TO_INTERVAL'])


#         weather_data = build_weather_service_data(weather_data, longitude, latitude, key)
#     return weather_data

In [14]:
# def build_weather_service_data(raw, longitude, latitude, service='CWB'):
#     data = raw.copy()
    
#     # read file and rename
#     if(service=='CWB'):
#         data['Location'] = data['LocationName'] + data['CityName']
#     elif(service=='IBM'):
#         data['Location'] = data['Name']
#     elif(service=='OWM'):
#         data['Location'] = data['Name']
        
#     data = data.drop_duplicates(['TIME_TO_INTERVAL', 'Location', 'TimeAhead'], keep="last")
    
#     # select data by hour
#     data['TIME_TO_INTERVAL'] = pd.to_datetime(data['TIME_TO_INTERVAL'])
#     if(service=='OWM'):
#         data['TIME_TO_INTERVAL'] = data['TIME_TO_INTERVAL']+ datetime.timedelta(hours=1)
    
#     data['Hour'] = pd.to_datetime(data['TIME_TO_INTERVAL']).dt.hour
#     data = data[data['Hour'].isin(range(6, 17+1))]
    
#     if(service=='CWB'):
#         data['WeatherType'] = data['WeatherType'].replace({
#             '午後短暫雷陣雨': '短暫陣雨或雷雨',
#             '有雨': '陣雨',
#             '短暫雨': '短暫陣雨',
#             '午後短暫陣雨': '短暫陣雨'})
#         data['WeatherType'] = data['WeatherType'].replace({
#             '短暫陣雨或雷雨': '陰',
#             '短暫陣雨': '陰',
#             '陣雨或雷雨': '陣雨'
#         })
        
#         pass
#     elif(service=='OWM'):
# # #         能見度
#         data['WeatherType'] = data['WeatherType'].replace({
#             '晴，少雲':'晴',
#             '多雲':'晴',
#             '陰，多雲':'晴',
#             '小雨':'晴',
#         })
#         #rename
#         data['WeatherType'] = data['WeatherType'].replace({
#             '小雪':'陣雨',
#             '大雨':'陰',
#             '中雨':'多雲',
#         })

#         pass
    
#     # find the recent forecast point, then get forecast data from recent point
#     locus = data.drop_duplicates(['Location'], keep="last").reset_index(drop=True)
#     recent = get_recent_target(longitude, latitude, locus)
#     print(recent)
#     data = data[data['Location'].eq(recent)].reset_index(drop=True)
    
#     # info output
#     print('服務預報點：', recent)
#     print(data.drop_duplicates(['WeatherType'], keep="last")['WeatherType'])
#     data = data.sort_values(by=['TIME_TO_INTERVAL', 'TimeAhead'])
#     return data 

In [15]:
# from math import radians, cos, sin, asin, sqrt
# # calculate distance based on latitude and longitude
# def geodistance(lon_a, lat_a, lon_b, lat_b):
#     lon_a, lat_a, lon_b, lat_b = map(radians, [lon_a, lat_a, lon_b, lat_b])
#     dlon = lon_b - lon_a
#     dlat = lat_b - lat_a
#     a = sin(dlat/2)**2 + cos(lat_a) * cos(lat_b) * sin(dlon/2)**2
#     dis = 2*asin(sqrt(a))*6371*1000
#     return dis
# def get_recent_target(longitude, latitude, locus, column='Location'):
#     # initialization
#     # recent_point: 表示距離輸入 plant 最近的 "預報點"，最終會被回傳
#     # shortest_dist: 表示表示距離輸入 station 最近 "預報點" 的距離，初始值設很大是為了避免一直寫入
#     recent_point = 'Not Found.'
#     shortest_dist = 1000*1000

#     # go through the search list
#     for i in range(len(locus)):
#         current = locus.loc[i]
#         dist = geodistance(
#             longitude, latitude, 
#             float(current['Longitude']), float(current['Latitude']))

#         # if the current distance is shorter than the historical shortest distance
#         # then use current point replace recent point
#         if(dist < shortest_dist):
#             shortest_dist = dist
#             recent_point = current[column]
#     # end of search, return
#     return recent_point

In [16]:
# plant_info = pd.read_csv('Plant_Info_Baoshan.csv')
# plant_info = plant_info.loc[1:1].reset_index(drop=True)
# # 案場資料
# latitude = plant_info['Latitude'][0]
# longitude = plant_info['Longitude'][0]
# #依據天氣類型去計算相似日的輻射值
# merge_data = calculate_similar_day(plant_info, latitude, longitude)

In [17]:
# merge_data = merge_data.reset_index(drop=True)

In [18]:
# merge_data.to_csv('./CWB.3H/Save/CWB.3H.Merge.Multiple.csv', index=None)

In [19]:
# d = pd.read_csv(f'./CWB.3H/Save/CWB.3H.Merge.Multiple.csv')

In [20]:
# d

In [21]:
# dd = pd.read_csv(f'./CWB.3H/Save/CWB.3H.Merge.Multiple(new)2.csv')

In [22]:
# dd

In [23]:
history = pd.read_csv(f'./CWB.3H/Save/CWB.3H.Merge.Multiple.csv')

C:\Users\GodZen\Anaconda3\envs\python3.6\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
history = history.sort_values(by=['TIME_TO_INTERVAL'], inplace=False)
history.tail(100)

,CityName,LocationName,geocode,Latitude,Longitude,WeatherType,WeatherType(index),ApparentTemperature(pred),Temperature(pred),RelativeHumidity(pred),...,PoP6h(pred),WindSpeed(pred),WindSpeed(index),WindDirection(pred),DewpointTemperature(pred),TIME_TO_INTERVAL,CrawlerTime,TimeAhead,Location,Hour
78832,彰化縣,和美鎮,10007030,24.110231,120.486077,晴,1,12,14,88,...,0.0,4,3,東北風,12.0,2023-01-19 06:00:00,2023-01-18 01:00:00,0,NaN,NaN
6338,彰化縣,和美鎮,10007030,24.110231,120.486077,晴,1,12,14,88,...,0.0,4,3,東北風,12.0,2023-01-19 06:00:00,2023-01-18 01:00:00,24,NaN,NaN
5070,彰化縣,北斗鎮,10007190,23.863301,120.513686,晴,1,11,12,98,...,0.0,2,2,偏北風,11.0,2023-01-19 06:00:00,2023-01-18 01:00:00,24,NaN,NaN
82642,彰化縣,埔鹽鄉,10007140,24.002038,120.455432,晴,1,12,13,93,...,0.0,4,3,偏北風,12.0,2023-01-19 06:00:00,2023-01-18 01:00:00,0,NaN,NaN
3802,彰化縣,伸港鄉,10007050,24.161568,120.478036,晴,1,12,14,87,...,0.0,4,3,東北風,12.0,2023-01-19 06:00:00,2023-01-18 01:00:00,24,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31169,彰化縣,線西鄉,10007040,24.132324,120.462580,晴,1,14,15,82,...,0.0,5,3,偏北風,12.0,2023-01-19 09:00:00,2023-01-18 01:00:00,24,NaN,NaN
98615,彰化縣,社頭鄉,10007170,23.898478,120.577712,晴,1,16,16,84,...,0.0,2,2,西北風,13.0,2023-01-19 09:00:00,2023-01-18 01:00:00,0,NaN,NaN
29901,彰化縣,竹塘鄉,10007250,23.862199,120.419721,晴,1,14,15,86,...,0.0,3,2,偏北風,12.0,2023-01-19 09:00:00,2023-01-18 01:00:00,24,NaN,NaN
38777,彰化縣,二水鄉,10007180,23.808358,120.610779,晴,1,16,15,84,...,0.0,2,<= 1,西北風,13.0,2023-01-19 09:00:00,2023-01-18 01:00:00,3,NaN,NaN
